In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd /content/drive/My\ Drive

/content/drive/My Drive


In [0]:
import re
import os
import codecs
import json
import csv
import spacy
import sklearn
import numpy as np
from time import time
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
!wget -O data.csv https://raw.githubusercontent.com/pratikpc/AIHackathon/fcknlp/data.csv?token=AKBFW4AOSTJP3QIUTBPHE525PTMQQ

--2019-09-07 12:12:06--  https://raw.githubusercontent.com/pratikpc/AIHackathon/fcknlp/data.csv?token=AKBFW4AOSTJP3QIUTBPHE525PTMQQ
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68529 (67K) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]  66.92K  --.-KB/s    in 0.02s   

2019-09-07 12:12:06 (4.34 MB/s) - ‘data.csv’ saved [68529/68529]



In [0]:
Data = "data.csv"
data = pd.read_csv(Data)
data.drop('Unnamed: 0', axis =1, inplace = True)
data.label = data.label.astype(int)
data = shuffle(data)
data['id'] = range(len(data))

lmtzr = WordNetLemmatizer()

In [0]:
print(data.groupby('label').size())

label
0    322
1    196
2    157
3    200
4    616
dtype: int64


In [0]:
data[data['label'] == 4]

<bound method DataFrame.count of                                                    text  label    id
573                         ford focus window wont roll      4     0
592                                    sign die battery      4     2
926                                chevy suburban light      4     3
897                                remove tire rim hand      4     8
658   get nissan fell glove compartment trunk open i...      4    13
1074                            system code come ranger      4    15
850                                                cold      4    18
909        chevrolet traverse oxygen sensor bank sensor      4    19
1044                                     best auto shop      4    20
600   grand cherokee sputter shuts back engine light...      4    21
917                                    oil engine burnt      4    22
1086                       part number miss back bumper      4    26
1091           fluid wheel tire leak brake fluid runner      4    28
6

In [0]:
label_3 = data[data['label'] == 3]
label_4 = data[data['label'] == 4]
label_3 = label_3[:300]
label_4 = label_4[:300]


In [0]:
with open('/content/drive/My Drive/bert/english.txt', 'r') as f:
  text = f.readlines()
print(len(text))
eng_words_ubuntu = set([lmtzr.lemmatize(x.strip().lower().replace('\'s', '')) for x in text] )
eng_dictionary_ubuntu = dict.fromkeys(eng_words_ubuntu, None)

102305


In [0]:
def is_english_word(word):
    try:
        x = eng_dictionary_ubuntu[word]
        return True
    except KeyError:
        return False
print(is_english_word("buy"))


def clean_words(x):
  res = []
  wrongs = []
  print(type(x), x)
  split = x.split()
  for word in split:
    word = word.strip()
    word =  lmtzr.lemmatize(word)
    if is_english_word(word) :
      res.append(word)
    else:
      wrongs.append(word)
  res_str = ' '.join(res)
  return res_str

data = data[data['text'].notnull()]
data['text'] = data['text'].apply(clean_words)
data = data.drop_duplicates(subset=['text','label'])
data = data[data['text'].notnull()]


In [0]:
text = data['text']
label = data['label']

In [0]:
xtrain, xtest = text[:int(np.ceil(len(text) * 0.80))], text[int(np.ceil(len(text) * 0.80)):]
ytrain, ytest = label[:int(np.ceil(len(label) * 0.80))], label[int(np.ceil(len(label) * 0.80)):]

In [0]:
def get_vectorizer(corpus, preprocessor=None, tokenizer=None):
    vectorizer = CountVectorizer()
    #vectorizer = CountVectorizer()
    vectorizer.fit(corpus)
    return vectorizer, vectorizer.get_feature_names()

def get_tfidf_vectorizer(corpus, preprocessor=None, tokenizer=None):
    vectorizer = TfidfVectorizer(ngram_range=(2,2))
    #vectorizer = CountVectorizer()
    vectorizer.fit(corpus)
    return vectorizer, vectorizer.get_feature_names()

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.utils.extmath import density


In [0]:
def data_for_training():
    vectorizer, feature_names = get_vectorizer(xtrain)
    
    X_train_no_HD = vectorizer.transform(xtrain).toarray()
    X_test_no_HD = vectorizer.transform(xtest).toarray()
            
    return X_train_no_HD, ytrain, X_test_no_HD, ytest, feature_names, vectorizer

def tfidf_for_training():
    vectorizer, feature_names = get_tfidf_vectorizer(xtrain)
    
    X_train_no_HD = vectorizer.transform(xtrain).toarray()
    X_test_no_HD = vectorizer.transform(xtest).toarray()
            
    return X_train_no_HD, ytrain, X_test_no_HD, ytest, feature_names, vectorizer

In [0]:
X_train, y_train, X_test, y_test, feature_names, vectorizer = data_for_training()

In [0]:
Xtrain, Ytrain, Xtest, Ytest, features, tfidf = tfidf_for_training()

In [0]:
'''def trim(s):
    """Trim string to fit on terminal (assuming 80-column display)"""
    return s if len(s) <= 80 else s[:77] + "..."


# #############################################################################
# Benchmark classifiers
def benchmark(clf, X_train, y_train, X_test, y_test, target_names,
              print_report=True, feature_names=None, print_top10=True,
              print_cm=True):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    clf.fit(X_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

    t0 = time()
    pred = clf.predict(X_test)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)

    score = metrics.accuracy_score(y_test, pred)
    print("accuracy:   %0.3f" % score)
    #print("Accuracy: %0.3f (+/- %0.3f)" % (score.mean(), score.std() * 2))

    if hasattr(clf, 'coef_'):
        print("dimensionality: %d" % clf.coef_.shape[1])
        print("density: %f" % density(clf.coef_))

        if print_top10 and feature_names is not None:
            print("top 10 keywords per class:")
            for i, label in enumerate(['New Car Enquiry','Test Drive Enquiry','Breakdown', 'Feedback', 'Quality']):
                top10 = np.argsort(clf.coef_[i])[-10:]
                print(trim("%s: %s" % (label, " ".join([feature_names[i] for i in top10]))))
        print()

    if print_report:
        print("classification report:")
        print(metrics.classification_report(y_test, pred,
                                            target_names=target_names))

    if print_cm:
        print("confusion matrix:")
        print(metrics.confusion_matrix(y_test, pred))
        #plot_confusion_matrix(metrics.confusion_matrix(y_test, pred), ['New Car Enquiry','Test Drive Enquiry','Breakdown', 'Feedback', 'Quality'])
        plt.show()

    print()
    clf_descr = str(clf).split('(')[0]
    return clf_descr, score, train_time, test_time'''

In [0]:
%matplotlib inline

In [0]:
clf_log_1 = LogisticRegression(C=1.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=20000,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
clf_log_1.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=20000,
                   multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
                   solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [0]:
print(metrics.classification_report(y_test, clf_log_1.predict(X_test)))

              precision    recall  f1-score   support

           0       0.50      0.44      0.47        54
           1       0.50      0.44      0.47        34
           2       0.85      0.46      0.59        24
           3       0.92      0.72      0.81        32
           4       0.79      0.94      0.86       140

    accuracy                           0.72       284
   macro avg       0.71      0.60      0.64       284
weighted avg       0.72      0.72      0.71       284



In [0]:
print(metrics.confusion_matrix(y_test, clf_log_1.predict(X_test)))

[[ 13   3   0  64   0]
 [  6  11   0  25   0]
 [  0   0   0  30   0]
 [  5   7   0 457   2]
 [  0   0   0 134  13]]


In [0]:

score = metrics.accuracy_score(y_test, clf_log_1.predict(X_test))
print("accuracy:   %0.3f" % score)

accuracy:   0.658
